# Deploy PyTorch Model using SageMaker and TorchServe

From the [AWS re:Invent 2020: Deploying PyTorch models for inference using TorchServe](https://www.youtube.com/watch?v=6xaMskcWmXY) talk

In [4]:
import boto3
import time
import json
from PIL import Image
import numpy as np
import sagemaker

sess = boto3.Session()
sm = sess.client('sagemaker')
region = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(boto_session=sess)

ClientError: An error occurred (ExpiredToken) when calling the GetCallerIdentity operation: The security token included in the request is expired

## Download DenseNet161 model from PyTorch model repository

In [5]:
!wget -q https://download.pytorch.org/models/densenet161-8d451a50.pth

^C


# Upload model and request handler file to Amazon S3

In [ ]:
bucket_name = sagemaker_session.default_bucket()
model_file_name = 'densenet161'
prefix = 'torchserve'

!mv densenet161-8d451a50.pth model.pth
!tar cvfz {model_file_name}.tar.gz model.pth model.py

In [ ]:
s3_model_path = sagemaker_session.upload_data(path=f'{model_file_name}.tar.gz', key_) # revise

In [ ]:
s3_model_path

## Create a PyTorch Model in Amazon SageMaker

In [ ]:
from sagemaker.pytorch.model import PyTorchPredictor
from sagemaker.predictor import Predictor
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(model_data=s3_model_path,
                     role=role,
                     entry_point='model.py',
                     framework_version='1.6.0',
                     py_version='py3',
                     predictor_cls=PyTorchPredictor)

## Deploy PyTorch Model to NVIDIA T4 using SageMaker model hosting

In [ ]:
%%time
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.g4dn.xlarge')

## Test model by generating predictions

In [ ]:
def image_preprocess(img):
    img = img.convert('RGB')
    img = np.asarray(img.resize((224,224)))

    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    img = (img / 255 - mean_vec) / stddev_vec

    img = np.expand_dims(img, axis=0)
    img = np.rollaxis(img, 3, 1)
    return img.astype(dtype='float32')

In [ ]:
!wget -q https://s3.amazonaws.com/model-server/inputs/kitten.jpeg

img = Image.open('kitten.jpeg')
img = image_preprocess(img)

response = predictor.predict(data=img)
pred = response[0].argsort()[-5:][::-1]

with open('imagenet_class_index.json', 'r') as f:
    labels = json.load(f)

for l in pred:
    print(labels[str(l)][1])